In [1]:
# Importe das bibliotecas necessárias
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dropout

In [2]:
# Configuração do caminho para o diretório do seu conjunto de dados
dataset_path = "C://Users//Pichau//Documents//GitHub//faro//IA//cats//images"

# Configuração do tamanho desejado para as imagens
image_size = (224, 224)
batch_size = 32


In [3]:
# Criação dos geradores de dados para treinamento
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

train_generator = train_datagen.flow_from_directory(
    dataset_path,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

Found 2692 images belonging to 22 classes.


In [4]:
# Criação dos geradores de dados para validação
validation_generator = train_datagen.flow_from_directory(
    dataset_path,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)


Found 666 images belonging to 22 classes.


In [5]:
# Carregamento da base MobileNetV2 pré-treinada, excluindo a camada superior
base_model = MobileNetV2(weights='imagenet', include_top=False)

# Adição de novas camadas para a classificação
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)  # Experimente diferentes taxas de dropout

num_classes = 22
predictions = Dense(num_classes, activation='softmax')(x)

# Criação do modelo final
model = Model(inputs=base_model.input, outputs=predictions)

# Congela as camadas do MobileNetV2 para que elas não sejam treinadas novamente
for layer in base_model.layers:
    layer.trainable = False

# Ajuste da taxa de aprendizado
custom_optimizer = Adam(learning_rate=0.0001)

# Compilação do modelo
model.compile(optimizer=custom_optimizer, loss='categorical_crossentropy', metrics=['accuracy'])


In [6]:
# Treinamento do modelo
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=60,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size
)

# Salvamento do modelo treinado
model.save('cat_breed_classifier.h5')


Epoch 1/50
84/84 [==============================] - 96s 1s/step - loss: 2.4599 - accuracy: 0.2992 - val_loss: 1.5319 - val_accuracy: 0.6109
Epoch 2/50
84/84 [==============================] - 67s 802ms/step - loss: 1.4624 - accuracy: 0.5688 - val_loss: 0.9923 - val_accuracy: 0.7344
Epoch 3/50
84/84 [==============================] - 68s 807ms/step - loss: 1.0956 - accuracy: 0.6564 - val_loss: 0.8341 - val_accuracy: 0.7359
Epoch 4/50
84/84 [==============================] - 64s 766ms/step - loss: 0.9176 - accuracy: 0.7105 - val_loss: 0.7133 - val_accuracy: 0.7656
Epoch 5/50
84/84 [==============================] - 64s 757ms/step - loss: 0.7951 - accuracy: 0.7451 - val_loss: 0.6816 - val_accuracy: 0.7922
Epoch 6/50
84/84 [==============================] - 64s 759ms/step - loss: 0.7185 - accuracy: 0.7718 - val_loss: 0.6090 - val_accuracy: 0.8078
Epoch 7/50
84/84 [==============================] - 63s 753ms/step - loss: 0.6491 - accuracy: 0.7895 - val_loss: 0.5859 - val_accuracy: 0.8000
Ep